# 18.369 pset 5

In [ ]:
# import a useful set of modules
from matplotlib import pyplot as plt
import numpy as np
import math
import meep as mp
from meep import mpb

## Problem 2

### Band diagram: Square lattice of rods

To start off with, here is some sample code computing the TM band diagram of a square lattice of dielectric rods in air.

In [ ]:
ms_sq = mpb.ModeSolver(
        resolution=32,
        num_bands=8,
    
        # a square lattice, period a=1:
        geometry_lattice=mp.Lattice(size=mp.Vector3(1, 1)),
        
        # cylindrical rods with radius 0.2a and epsilon=12
        geometry=[mp.Cylinder(0.2, material=mp.Medium(epsilon=12))],
    
        # the corners of the irreducible Brillouin zone
        k_points = mp.interpolate(9, [
            mp.Vector3(),               # Gamma
            mp.Vector3(0.5, 0),         # X
            mp.Vector3(0.5, 0.5),       # M
            mp.Vector3(),               # Gamma
        ]))
ms_sq.run_tm()

Plotting the result and computing the gap:

In [ ]:
plt.figure(figsize=(10,5))

md = mpb.MPBData(rectify=True, periods=3)
eps = md.convert(ms_sq.get_epsilon())
plt.subplot(1,2,2)
plt.imshow(eps, cmap="binary")
plt.title("$example: \epsilon$ for square lattice of rods")

plt.subplot(1,2,1)
plt.plot(ms_sq.all_freqs, "b.-")
plt.ylabel("frequency ($c/a$)")
plt.xticks([0,10,20,30], ["$\Gamma$","X","M","$\Gamma$"])
plt.grid()
plt.title("example: TM square-lattice bands")

band1_max = np.max(ms_sq.all_freqs[:,0])
band2_min = np.min(ms_sq.all_freqs[:,1])
gap = 200 * (band2_min - band1_max) / (band1_max + band2_min)
print("TM band gap = %g%%" % gap)

### Starting code: Transmission through $N_x$ rods

The following function computes the transmitted power $P(\omega)$ through a finite (in $x$) crystal consisting of $N_x$ rods, with air on either side, returning an array of frequencies, an array of powers, and the simulation object:

In [ ]:
def transmitted_P(Nx=5, pad=4, dpml=1, fmin=0.2, fmax=0.8, nfreq=300):
    fcen = (fmin+fmax)*0.5
    df = fmax - fmin
    sx = Nx + 2*(pad+dpml)
    
    sim = mp.Simulation(cell_size=mp.Vector3(sx, 1),
                        
                        # make Nx duplicates of a rod:
                        geometry = mp.geometric_object_duplicates(mp.Vector3(1,0), 0, Nx-1, 
                                                                  mp.Cylinder(0.2, center=mp.Vector3(-0.5*sx+dpml+pad+0.5,0), material=mp.Medium(epsilon=12))),
 
                        
                        # absorbing boundaries in y direction, periodic in x
                        boundary_layers = [mp.PML(dpml, direction=mp.X)],
                        k_point = mp.Vector3(0,0), # ky=0 for normal incidence
                        
                        # source is a pulsed line source adjacent to PML
                        sources=[mp.Source(mp.GaussianSource(fcen, fwidth=df), mp.Ez,
                                           center=(-sx*0.5 + dpml,0), size=(0,1))],
                        
                        # to speed things up, exploit the y=0 mirror plane:
                        symmetries = [mp.Mirror(mp.Y, phase=1)],
                        resolution=20)
    
    # flux (power) monitor on transmitted side
    fluxregion = mp.FluxRegion(center=mp.Vector3(0.5*sx-dpml),
                               size=mp.Vector3(0,1))
    trans = sim.add_flux(fcen, df, nfreq, fluxregion)
    
    # run simulation until fields on transmitted side have decayed away
    sim.run(until_after_sources=mp.stop_when_fields_decayed(50, mp.Ez, mp.Vector3(0.5*sx-dpml), 1e-3))
    return sim, np.array(mp.get_flux_freqs(trans)), np.array(mp.get_fluxes(trans))

Let's run it for $N_x = 5$ layers of the crystal:

In [ ]:
sim,freqs,P5 = transmitted_P(Nx=5)

As usual, it is always a good idea to visualize the simulation, to make sure you are running what you intend.  The Meep simulation object `sim` has a method `sim.plot2D()` that shows both the dielectric function ε (black circles), the PML regions (green striped boxes), the source location (red line) and the flux-monitor location (green line):

In [ ]:
sim.plot2D()

We can also plot power (Poynting flux) measured through the transmission plane vs. frequency, but this is **not yet normalized**.  That is, we are plotting *absolute* power (in arbitrary units), whereas what we eventually want is to plot the *fraction of the incident power* that is transmitted.

In [ ]:
plt.plot(freqs, P5)
plt.xlabel("frequency ($c/a$)")
plt.ylabel("transmitted power (arb. units)")
plt.title("unnormalized transmitted power")

Basically, it is the incident Gaussian pulse spectrum multiplied by the "wiggly" transmission spectrum.

It is also nice to visualize the simulation in the form of an animation of the $E_z$ field, which is accomplished using the `Animate2D` object provided by Meep:

In [ ]:
sim.reset_meep()
animate = mp.Animate2D(sim, fields=mp.Ez)
sim.run(mp.at_every(0.2,animate),until=50)
animate.to_jshtml(10)

### part (a)

Compute the *normalized* transmission spectra for frequencies from $0.2c/a$ to $0.8c/a $as a function of $N_{x}$, for $N_{x}=1,2,3,5,6$, and plot them (together on a single plot). The transmission spectrum is the transmitted power spectrum (shown above) *normalized* by dividing by the transmitted power for $N_{x}=0$ (no holes).

Try to explain the features of your transmission spectrum to the band diagram in the sample code.

### part (b)

Predict analytically at what frequency $\omega_{0}$ you should start to see additional diffracted orders in the reflected wave (i.e. reflected waves at angles in addition to the normal 0° reflection).

Now, modify the simulation to use a TM continuous-wave (CW) source and output $E_{z}$ at the end and show that there is a qualitative change in the reflected field pattern if you put in a frequency just below $\omega_{0}$ versus a frequency just above $\omega_{0}$. 